# Project: Summarization App Using LangChain and OpenAI

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

### A) Basic Prompt

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)


In [9]:
text= """
Mojo combines the usability of Python with the performance of C, unlocking unparalleled programmability \
of AI hardware and extensibility of AI models.
Mojo is a new programming language that bridges the gap between research and production \ 
by combining the best of Python syntax with systems programming and metaprogramming.
With Mojo, you can write portable code that’s faster than C and seamlessly inter-op with the Python ecosystem.
When we started Modular, we had no intention of building a new programming language. \
But as we were building our platform with the intent to unify the world’s ML/AI infrastructure, \
we realized that programming across the entire stack was too complicated. Plus, we were writing a \
lot of MLIR by hand and not having a good time.
And although accelerators are important, one of the most prevalent and sometimes overlooked "accelerators" \
is the host CPU. Nowadays, CPUs have lots of tensor-core-like accelerator blocks and other AI acceleration \
units, but they also serve as the “fallback” for operations that specialized accelerators don’t handle, \
such as data loading, pre- and post-processing, and integrations with foreign systems. \
"""

messages = [
    SystemMessage(content='You are an expert copywriter with expertize in summarizing documents'),
    HumanMessage(content=f'Please provide a short and concise summary of the following text:\n TEXT: {text}')
]

llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')



In [4]:
llm.get_num_tokens(text)

231

In [5]:
summary_output = llm(messages)

In [6]:
print(summary_output.content)

Mojo is a new programming language that combines the usability of Python with the performance of C, allowing for unparalleled programmability of AI hardware and extensibility of AI models. It bridges the gap between research and production by offering portable code that is faster than C and seamlessly interoperates with the Python ecosystem. Mojo was created to simplify programming across the entire ML/AI infrastructure stack, including the host CPU, which is often overlooked as an accelerator for operations that specialized accelerators cannot handle.


### Summarizing Using Prompt Templates

In [7]:
from langchain import PromptTemplate
from langchain.chains import LLMChain

In [13]:
template = '''
Write a concise and short summary of the following text:
TEXT: `{text}`
Translate the summary to {language}.
'''
prompt = PromptTemplate(
    input_variables=['text', 'language'],
    template=template
)

In [10]:
llm.get_num_tokens(promp.format(text=text, language='English'))

251

In [19]:
chain = LLMChain(llm=llm, prompt=prompt)
summary = chain.run({'text': text, 'language':'hindi'})

In [20]:
print(summary)

Mojo एक नया प्रोग्रामिंग भाषा है जो Python की उपयोगिता को C की प्रदर्शन के साथ मिलाकर AI हार्डवेयर की अद्वितीय प्रोग्रामिंग और AI मॉडल की विस्तारयोग्यता को खोलता है। Mojo एक नई प्रोग्रामिंग भाषा है जो Python सिंटैक्स का उपयोग करते हुए सिस्टम प्रोग्रामिंग और मेटाप्रोग्रामिंग का संयोजन करती है। Mojo के साथ, आप C से भी तेज़ पोर्टेबल कोड लिख सकते हैं और Python एकोसिस्टम के साथ सहजता से इंटर-ऑप कर सकते हैं।


### Summarizing using SuffDocumentChain

In [21]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document


In [28]:
with open('files/sj.txt', encoding='utf-8') as f:
    text = f.read()

# text

docs = [Document(page_content=text)]
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [29]:
template = '''Write a concise and short summary of the following text.
TEXT: `{text}`
'''
prompt = PromptTemplate(
    input_variables=['text'],
    template=template
)

In [30]:
chain = load_summarize_chain(
    llm,
    chain_type='stuff',
    prompt=prompt,
    verbose=False
)
output_summary = chain.run(docs)

In [31]:
print(output_summary)

Steve Jobs, in his commencement speech at Stanford University, shares three stories from his life. The first story is about connecting the dots and how dropping out of college led him to take a calligraphy class, which later influenced the design of the Macintosh computer. The second story is about love and loss, where he talks about being fired from Apple and how it led him to start over and create successful companies like NeXT and Pixar. The third story is about death, where he talks about his battle with cancer and how it taught him to live each day as if it were his last. He ends his speech by encouraging the graduates to stay hungry and stay foolish.


### Summarizing Large Documents Using map_reduce

In [32]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [33]:
with open('files/sj.txt', encoding='utf-8') as f:
    text = f.read()

llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [34]:
llm.get_num_tokens(text)

2653

In [38]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=50)
chunks = text_splitter.create_documents([text])

In [39]:
len(chunks)

2

In [40]:
chain = load_summarize_chain(
    llm,
    chain_type='map_reduce',
    verbose=False
)
output_summary = chain.run(chunks)

In [41]:
print(output_summary)

Steve Jobs' commencement speech at Stanford University shares three stories from his life, emphasizing the importance of connecting the dots, love and loss, and living each day as if it were your last. He encourages graduates to follow their heart and intuition, be courageous, and stay hungry and foolish. He references The Whole Earth Catalog and wishes the same for the graduates.


In [42]:
chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [43]:
chain.combine_document_chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

### map_reduce wich Custom Prompts

In [44]:
map_prompt = '''
Write a short and concise summary of the following:
Text: `{text}`
CONCISE SUMMARY:
'''
map_prompt_template = PromptTemplate(
    input_variables=['text'],
    template=map_prompt
)

In [45]:
combine_prompt = '''
Write a concise summary of the following text that covers the key points.
Add a title to the summary.
Start your summary with an INTRODUCTION PARAGRAPH that gives an overview of the topic FOLLOWED
by BULLET POINTS if possible AND end the summary with a CONCLUSION PHRASE.
Text: `{text}`
'''
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=['text'])

In [46]:
summary_chain = load_summarize_chain(
    llm=llm,
    chain_type='map_reduce',
    map_prompt=map_prompt_template,
    combine_prompt=combine_prompt_template,
    verbose=False
)
output = summary_chain.run(chunks)

In [48]:
print(output)

Steve Jobs' Commencement Speech at Stanford University

- Steve Jobs shares three stories from his life in his commencement speech at Stanford University.
- The first story is about connecting the dots and how dropping out of college led him to take a calligraphy class that later influenced the design of the Macintosh computer.
- The second story is about love and loss, where he talks about being fired from Apple and how it led him to start new companies and find love.
- The third story is about death, where he shares his experience of being diagnosed with cancer and how it made him realize the importance of living each day as if it were his last.
- Death is inevitable and serves as a change agent for life.
- Don't waste your limited time living someone else's life or being trapped by dogma.
- Follow your heart and intuition.
- The Whole Earth Catalog had a farewell message on its final issue: "Stay Hungry. Stay Foolish."
- The speaker wishes the same for the graduates. 

Conclusion: S

### Summarizing Using the refine Chain

In [49]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredPDFLoader

In [50]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [53]:
pip install unstructured -q

Note: you may need to restart the kernel to use updated packages.


In [55]:
pip install pdf2image

In [56]:
loader = UnstructuredPDFLoader('files/attention_is_all_you_need.pdf')
data = loader.load()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ad\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ad\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [58]:
# print(data[0].page_content)

In [60]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [61]:
len(chunks)

4

In [62]:
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [63]:
def print_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('gpt-3.5-turbo')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.002:.6f}')
    
    
print_embedding_cost(chunks)

Total Tokens: 9415
Embedding Cost in USD: 0.018830


In [66]:
chain = load_summarize_chain(
    llm=llm,
    chain_type='refine',
    verbose=True
)
output_summary = chain.run(chunks)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"7 1 0 2 c e D 6

] L C . s c [

5 v 2 6 7 3 0 . 6 0 7 1 : v i X r a

Attention Is All You Need

Ashish Vaswani∗ Google Brain avaswani@google.com

Noam Shazeer∗ Google Brain noam@google.com

Niki Parmar∗ Google Research nikip@google.com

Jakob Uszkoreit∗ Google Research usz@google.com

Llion Jones∗ Google Research llion@google.com

Aidan N. Gomez∗ † University of Toronto aidan@cs.toronto.edu

Łukasz Kaiser∗ Google Brain lukaszkaiser@google.com

Illia Polosukhin∗ ‡ illia.polosukhin@gmail.com

Abstract

The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, 


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary
We have provided an existing summary up to a certain point: The paper proposes a new neural network architecture called the Transformer, which is based solely on attention mechanisms and does not use recurrent or convolutional neural networks. The Transformer achieves superior results in machine translation tasks while being more parallelizable and requiring less training time. The model achieves a BLEU score of 28.4 on the WMT 2014 English-to-German translation task and a state-of-the-art BLEU score of 41.8 on the WMT 2014 English-to-French translation task. The paper also discusses the advantages of self-attention and describes the attention mechanism used in the Transformer.
We have the opportunity to refine the existing summary(only if needed) with some more context below.
------------
Attention(Q, K, V ) = softmax(

QK T √ dk

)V

(1)

1√

The two most commonly use


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary
We have provided an existing summary up to a certain point: The paper introduces a new neural network architecture called the Transformer, which relies solely on attention mechanisms and does not use recurrent or convolutional neural networks. The Transformer achieves superior results in machine translation tasks, with a BLEU score of 28.4 on the WMT 2014 English-to-German translation task and a state-of-the-art BLEU score of 41.8 on the WMT 2014 English-to-French translation task. The paper also discusses the advantages of self-attention and describes the attention mechanism used in the Transformer. The model uses multi-head attention in three different ways: encoder-decoder attention layers, self-attention layers in the encoder, and self-attention layers in the decoder. The Transformer also employs position-wise feed-forward networks, embeddings and softmax, and posit


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary
We have provided an existing summary up to a certain point: The Transformer is a new neural network architecture that relies solely on attention mechanisms and does not use recurrent or convolutional neural networks. It achieves superior results in machine translation tasks, with a state-of-the-art BLEU score of 41.8 on the WMT 2014 English-to-French translation task. The model uses multi-head attention in three different ways and employs position-wise feed-forward networks, embeddings and softmax, and positional encoding. The paper also discusses the advantages of self-attention and describes the attention mechanism used in the Transformer. The training regime for the models is described, including the use of the Adam optimizer and three types of regularization. The Transformer generalizes well to English constituency parsing, outperforming all previously reported mode


> Finished chain.

> Finished chain.


In [65]:
print(output_summary)

The Transformer is a neural network architecture that relies solely on attention mechanisms and achieves superior results in machine translation tasks while being more parallelizable and requiring less training time than architectures based on recurrent or convolutional layers. The model consists of stacked self-attention and point-wise, fully connected layers for both the encoder and decoder. The attention mechanism used is called Scaled Dot-Product Attention. The paper also compares self-attention layers to recurrent and convolutional layers in terms of computational complexity, parallelizability, and path length between long-range dependencies in the network. The Transformer generalizes well to English constituency parsing, yielding better results than all previously reported models with the exception of the Recurrent Neural Network Grammar. The authors plan to apply attention-based models to other tasks and extend the Transformer to handle input and output modalities other than tex

### refine With Custom Prompts

In [68]:
prompt_template = """Write a concise summary of the following extracting the key information:
Text: `{text}`
CONCISE SUMMARY:"""
initial_prompt = PromptTemplate(template=prompt_template, input_variables=['text'])

refine_template = '''
    Your job is to produce a final summary.
    I have provided an existing summary up to a certain point: {existing_answer}.
    Please refine the existing summary with some more context below.
    ------------
    {text}
    ------------
    Start the final summary with an INTRODUCTION PARAGRAPH that gives an overview of the topic FOLLOWED
    by BULLET POINTS if possible AND end the summary with a CONCLUSION PHRASE.
    
'''
refine_prompt = PromptTemplate(
    template=refine_template,
    input_variables=['existing_answer', 'text']
)


In [69]:
chain = load_summarize_chain(
    llm=llm,
    chain_type='refine',
    question_prompt=initial_prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=False
    
)
output_summary = chain.run(chunks)

In [70]:
print(output_summary)

Introduction:
The Transformer is a neural network architecture that uses attention mechanisms to achieve state-of-the-art results in machine translation tasks while requiring significantly less training time. It replaces recurrent layers with multi-headed self-attention, allowing for more parallelization and faster training times.

Key Points:
- The Transformer achieves a new state of the art in machine translation tasks, outperforming even previously reported ensembles.
- The model can be trained significantly faster than architectures based on recurrent or convolutional layers.
- The Transformer's attention mechanisms and self-attention layers allow for more parallelization and faster training times, making it a promising approach for a wide range of natural language processing tasks.
- The Transformer's ability to generalize well to other tasks, such as English constituency parsing, highlights its potential for broader applications in the field.
- Attention visualizations show that 

### Summarizing Using LangChain Agents

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.utilities import WikipediaAPIWrapper

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [3]:
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')
wikipedia = WikipediaAPIWrapper()

In [4]:
tools = [
    Tool(
        name="Wikipedia", 
        func=wikipedia.run,
        description="Useful for when you need to get information from wikipedia about a single topic"
    )
]

In [5]:
agent_executor = initialize_agent(tools, llm, agent='zero-shot-react-description', verbose=True)

In [6]:
output = agent_executor.run('Can you please provide a short summary of George Washington?')



> Entering new AgentExecutor chain...
I should use Wikipedia to get a brief summary of George Washington's life and accomplishments.
Action: Wikipedia
Action Input: "George Washington"
Observation: Page: George Washington
Summary: George Washington (February 22, 1732 – December 14, 1799) was an American military officer, statesman, and Founding Father who served as the first president of the United States from 1789 to 1797. Appointed by the Continental Congress as commander of the Continental Army, Washington led Patriot forces to victory in the American Revolutionary War and served as president of the Constitutional Convention of 1787, which created and ratified the Constitution of the United States and the American federal government. Washington has been called the "Father of his Country" for his manifold leadership in the nation's founding.Washington's first public office, from 1749 to 1750, was as surveyor of Culpeper County, Virginia. He subsequently received his first military 

In [7]:
print(output)

George Washington was an American military officer, statesman, and Founding Father who served as the first president of the United States from 1789 to 1797. He led Patriot forces to victory in the American Revolutionary War and served as president of the Constitutional Convention of 1787, which created and ratified the Constitution of the United States and the American federal government. He was also a slave owner who had a complicated relationship with slavery. George Washington Carver was an American agricultural scientist and inventor who promoted
